### Maxcut Runtime

### Preparing Execution

In [ ]:
min_qubits=4
max_qubits=6
max_circuits=1
num_shots=5000

degree = 3
rounds = 1
max_iter = 30
parameterized = False

max_execution_time = 100_000 # seconds

backend_id="ibmq_qasm_simulator"
# hub="ibm-q"; group="open"; project="main"
hub="ibm-q-research"; group="quantum-circ-1"; project="main"
provider_backend = None
exec_options = None

API_TOKEN = "YOUR API TOKEN"

# # *** If using IBMQ hardware, run this once to authenticate
# from qiskit import IBMQ
# IBMQ.save_account(API_TOKEN)

# # *** If you are part of an IBMQ group, set hub, group, and project name here
# hub="YOUR_HUB_NAME"
# group="YOUR_GROUP_NAME"
# project="YOUR_PROJECT_NAME"

# # *** This example shows how to specify the backend using a known "backend_id"
# # Use 'sabre' layout for IBM backends
# exec_options = { "optimization_level":3, "layout_method":'sabre', "routing_method":'sabre' }
# backend_id="ibmq_belem"

# # *** Here's an example of using a typical custom provider backend (e.g. AQT simulator)
# import os
# from qiskit_aqt_provider import AQTProvider
# provider = AQTProvider(os.environ.get('AQT_ACCESS_KEY'))    # get your key from environment
# provider_backend = provider.backends.aqt_qasm_simulator_noise_1
# backend_id="aqt_qasm_simulator_noise_1"

# # An example using IonQ provider
# from qiskit_ionq import IonQProvider
# provider = IonQProvider()   # Be sure to set the QISKIT_IONQ_API_TOKEN environment variable
# provider_backend = provider.get_backend("ionq_qpu")
# backend_id="ionq_qpu"

# # *** Use these settings for better results
# min_qubits=4
# max_qubits=10
# max_circuits=2
# num_shots=50000

In [ ]:
import uuid
from qiskit_ibm_runtime import QiskitRuntimeService
import runtime_utils

# Creating runtime script to be uploaded
RUNTIME_FILENAME = 'maxcut_runtime.py'
runtime_utils.create_runtime_script(file_name=RUNTIME_FILENAME)

# Meta data required by qiskit runtime
meta = {
  "name": f"qedc-maxcut-benchmark-{uuid.uuid4()}",
  "description": "A sample Maxcut Benchmark program.",
  "max_execution_time": 100_000,
  "version": "1.0",
}

service = QiskitRuntimeService(
    channel='ibm_quantum',
    token=API_TOKEN
)

program_id = service.upload_program(data=RUNTIME_FILENAME, metadata=meta)

### Executing benchmark on Runtime Servers

In [ ]:
from qiskit import IBMQ
import json

import traceback
import os


IBMQ.load_account()

provider = IBMQ.get_provider(hub=hub, group=group, project=project)

options = {
    'backend_name': backend_id
}

with open(os.path.join("..", "_common", "instances.json"), "r") as file:
    insts = json.load(file)

runtime_inputs = {
    "backend_id": backend_id,
    "method": 2,
    "_instances": insts,
    "min_qubits": min_qubits,
    "max_qubits": max_qubits,
    "max_circuits": max_circuits,
    "num_shots": num_shots,

    "degree": degree,
    "rounds": rounds,
    "max_iter": max_iter,
    "parameterized": parameterized
}

try:
    job = service.run(
        program_id=program_id,
        options=options,
        inputs=runtime_inputs,
        instance=f'{hub}/{group}/{project}'
    )

    print(f'{job.creation_date.ctime() = }')
    print(f'{job.job_id = }')
    print(f'{job.program_id = }')

except Exception as e:
    print(f'The Exception {e} has occurred!')
    # Prints full traceback for easier debugging
    print(traceback.format_exc())

finally:
    IBMQ.disable_account()

job_id = job.job_id
%store job_id

### Get result directly from job

In [ ]:
# Get results

result = job.result()

### Get result from `job_id`

In [ ]:
retrieved_result = provider.runtime.job(job.job_id)
result = retrieved_result.result()

### Save results to file

In [ ]:
import maxcut_benchmark
maxcut_benchmark.save_runtime_data(result)

In [ ]:
maxcut_benchmark.load_data_and_plot('__data')